# Generación de tonos para detectores MKID
## Condición $K=M, n_0=0, k_0=0$  y BF-DFTG 

Por detalles, ver mi tesis ;)

Esta condición corresponde al apilamiento **par** y es lo mismo que el BF-DFT críticamente muetreado.

### Diseño
Este script genera y grafica las señales que se van a enviar en cada canal. 

La idea es simular la parte del sintetizador y del analizador para el canalizador de 16 canales. 

Importo los modulos necesarios

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lfilter, kaiser
from scipy.fftpack import fft, ifft, fftshift, fftfreq
from dsp_funcs import h_freqz, mfreqz

Configuración.

In [ ]:
Fs            = 125e6    #Sampling frequency
M             = 16       # downsampling ratio
I             = 1        # oversampling factor
K             = M*I      # number of channels
k0            = 0        # 
n0            = 0
data_len       = 2**16;
prototipo      = '../coefficients/coeff/prototipo.csv';
write_golden_data = 1;

Creo las funciones que me generan las señales teniendo en cuenta las fases de Newman. Para detalles, ver el notebook 0_crest_factor_analysis.ipynb.

In [ ]:
def newman_sig(N,dlen,N0=1):
    nsig = np.zeros((N,dlen),dtype=complex)
    n=np.arange(dlen)
    for k in range(1,N+1):
        nsig[k-1] = np.exp(1j*(((k+N0)/(2*N*np.pi))*n+(np.pi*(k-1)**2)/N))
        fk=Fs*(k+N0)/(4*np.pi**2*N)
        print(fk)
    return np.sqrt(2/N)*nsig
    #return nsig

def newman_sig_rand(N,dlen,N0=1):
    nsig = np.zeros((N,dlen),dtype=complex)
    n=np.arange(dlen)
    for k in range(1,N+1):
        nsig[k-1] = np.random.randn()*np.exp(1j*(((k+N0)/(2*N*np.pi))*n+(np.pi*(k-1)**2)/N))
    return np.sqrt(2/N)*nsig
    #return nsig

Esta parte es para ver los tonos generados en función del tiempo.

In [ ]:
print('Create input data....');
#data_type = 'constant';
data_type = 'cosine';

#np.random.seed(131) #replicable results
n=np.arange(1,data_len+1)
tx_data_in = np.zeros((K,data_len),dtype=complex)
#for k in range(1,K+1):
#    if 'constant' in data_type:
#        tx_data_in[k-1] = np.sqrt(2/K)*np.cos(2*np.pi*((k+1)/(2*K*np.pi))*n) #newman_sig(16,data_len)
        #tx_data_in_b[k-1] = np.random.randn()*newman_sig(k,data_len)
#    else:
#        tx_data_in[k-1] = np.sqrt(2/K)*np.exp(1j*((k+1)/(2*K*np.pi))*n) #newman_sig(16,data_len)
        
#        if k in [1,2,4,5,7,8,9,10,11,12,13,15,16]:
#            tx_data_in[k-1] = np.zeros(data_len,dtype=complex)

    
#tx_data_in = newman_sig(K,data_len)
tx_data_in = newman_sig(K,data_len)+ 0.0017 * np.random.randn(data_len) #agrego ruido para pruebas
#tx_data_in = newman_sig(K,data_len)+ 0.0017 * np.random.normal(0,1,size=data_len) #agrego ruido para pruebas
for k in range(1,K+1):
    if k in [1,2,4,5,7,8,9,10,11,12,13,15,16]:
        tx_data_in[k-1] = np.zeros(data_len,dtype=complex)

#tx_data_in_b = newman_sig_rand(50,data_len)
#for i,j in enumerate(tx_data_in_b):
#    tx_data_in_b[i] = np.random.randn()*j

Creamos los datos de entrada

**Constante** o un **coseno** (escalado por el numero de canal) en cada canal, excluyendo el 4to, 6to y 14vo canal con propósito de demostración 

In [ ]:
#print('Create input data....');
#data_type = 'constant';
#data_type = 'cosine';

#n=np.arange(1,data_len+1)
#tx_data_in = np.zeros((K,data_len),dtype=complex)
#Creo las señales modulantes y elimino alguno de los canales para hacer pruebas
#for k in range(1,K+1):
#    if 'constant' in data_type:
#        tx_data_in[k-1] = np.cos(((2*np.pi*100e3*k)/(125e6))*n)
#    else:
#        #tx_data_in[k-1] = np.exp(1j*(k/(K*2*np.pi))*n)
#        #tx_data_in[k-1] = np.exp(2j*np.pi*((200e3/Fs)*k*n))
#        tx_data_in[k-1] = np.exp(2j*np.pi*((200e3/Fs)*k*n+np.random.random()))
#        #tx_data_in[k-1] = np.random.random()*np.exp(2j*np.pi*((200e3/Fs)*k*n+np.random.random()))

#        #if (k == 4 or k==6 or k==14):
#        #if k in np.arange(1,16):
#        if k in [1,2,4,5,7,8,9,10,11,12,13,15,16]:
#            tx_data_in[k-1] = np.zeros(data_len,dtype=complex)

Esta parte es para ver los tonos generados en función del tiempo.

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches((10,5))
#fig.suptitle('Input Spectrum',fontsize=18)
for i in tx_data_in:
    ax.plot(n-1,i.real)
    #ax.pause(0.3)

ax.plot(np.arange(data_len),np.sum(tx_data_in.real,axis=0),'k',lw=2)
ax.set_xlim(0,2**10)
#ax.set_ylim(-17.5,2)
ax.grid()
#ax.legend(loc=0,fontsize=10)
#ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
ax.set_ylabel(r'Amplitud (a.u.)',fontsize=12)
ax.set_xlabel('Indice de tiempo n',fontsize=12)
plt.savefig('plots/in_spectrum_time.pdf')
plt.show()

Esta parte es donde grafico el espectro de las señales de salida

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
#fig.set_size_inches((8,5))
#fig.suptitle('Input Spectrum',fontsize=18)

NFFT=2**13
ww = kaiser(data_len, beta=8)
X=fft(tx_data_in*ww,NFFT)
freq=fftfreq(NFFT,d=1/Fs)
for n in range(K):
    ax.plot(freq[:int(NFFT/2)],20*np.log10(abs(X[n][:int(NFFT/2)])/max(abs(X[n][:int(NFFT/2)]))),label='Ch{}'.format(n))

ax.set_xlim(0,Fs/2)
ax.set_ylim(-90,2)
ax.grid()
#ax.legend(loc=0,fontsize=10)
ax.legend(loc='upper right', fontsize='small', ncol=2)
ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
ax.set_ylabel(r'Amplitud (dB)',fontsize=12)
ax.set_xlabel('Frecuencia (Hz)',fontsize=12)
plt.tight_layout()
plt.savefig('plots/in_spectrum.pdf')
plt.show()

Guardo los archivos con los datos de los canales.

In [ ]:
if write_golden_data:
    np.savetxt('../golden_data/tx_source_real_{}.dat'.format(data_type),np.reshape(tx_data_in.real,(K*data_len),order='F'),delimiter='\n',fmt='%.32e')
    np.savetxt('../golden_data/tx_source_imag_{}.dat'.format(data_type),np.reshape(tx_data_in.imag,(K*data_len),order='F'),delimiter='\n',fmt='%.32e')

Ahora generamos el archivo tx_source_real/imag_{data_type}_{K}.coe para cargar las memorias de datos en Vivado

In [ ]:
data_in=np.reshape(tx_data_in.real,(1,K*data_len),order='F')
c_frac_width      = 15 # data fractional width

f=open('../coefficients/coeff/tx_source_real_{}_{}_check.coe'.format(data_type,K),'w')
for i in range(tx_data_in.size):
    if (int(data_in[0][i]*2**c_frac_width) < 0):
        f.write('{:x}'.format(int(data_in[0][i]*2**c_frac_width+2**(c_frac_width+1))))
    else:
        f.write('{:x}'.format(int(data_in[0][i]*2**c_frac_width)))
    if i == data_in.size-1:
        continue
        #f.write(';')
    else:
        f.write(',')
f.close()

f=open('../coefficients/coeff/tx_source_real_{}_{}.coe'.format(data_type,K),'w')
#f.write('memory_initialization_radix=10;\n')
f.write('memory_initialization_radix=16;\n')
f.write('memory_initialization_vector=')
for i in range(tx_data_in.size):
    if (int(data_in[0][i]*2**c_frac_width) < 0):
        f.write('{:x}'.format(int(data_in[0][i]*2**c_frac_width+2**(c_frac_width+1))))
    else:
        f.write('{:x}'.format(int(data_in[0][i]*2**c_frac_width)))
    if i == data_in.size-1:
        f.write(';')
    else:
        f.write(',')
f.close()

data_in=np.reshape(tx_data_in.imag,(1,K*data_len), order='F')

f=open('../coefficients/coeff/tx_source_imag_{}_{}_check.coe'.format(data_type,K),'w')
for i in range(tx_data_in.size):
    if (int(data_in[0][i]*2**c_frac_width) < 0):
        f.write('{:x}'.format(int(data_in[0][i]*2**c_frac_width+2**(c_frac_width+1))))
    else:
        f.write('{:x}'.format(int(data_in[0][i]*2**c_frac_width)))
    if i == data_in.size-1:
        continue
        #f.write(';')
    else:
        f.write(',')
f.close()

f=open('../coefficients/coeff/tx_source_imag_{}_{}.coe'.format(data_type,K),'w')
#f.write('memory_initialization_radix=10;\n')
f.write('memory_initialization_radix=16;\n')
f.write('memory_initialization_vector=')
for i in range(tx_data_in.size):
    if (int(data_in[0][i]*2**c_frac_width) < 0):
        f.write('{:x}'.format(int(data_in[0][i]*2**c_frac_width+2**(c_frac_width+1))))
    else:
        f.write('{:x}'.format(int(data_in[0][i]*2**c_frac_width)))
    if i == data_in.size-1:
        f.write(';')
    else:
        f.write(',')
f.close()

# Sintetizador FDM

Lo que hago a partir de acá es simular todo el sistema transmisor en python para luego poder comparar con lo que obtengo en mis simulaciones con Vivado y los cores generados.

Recuperar el filtro prototipo del channelizer.

In [ ]:
proto = np.loadtxt(prototipo,delimiter=',');

Reshape de los coeficientes del filtro en filtros polifásicos es decir, dividir en las **$K$** polifases.

Acá es donde armo los filtros polifásicos: $q_\rho(m) = h(mM+\rho),\quad \rho=0,1,...,M-1,\quad K=M$.

In [ ]:
Fp=np.zeros((K,int(proto.size/K)))

In [ ]:
for ro in range(K):
    for m in range(int(proto.size/K)):
        Fp[ro][m] = proto[int(ro+M*m)]
#print(Fp)
#proto[2][100]
#for ro in range(1,K):
#    Fp[ro] = np.roll(Fp[ro],-1) 

Para graficar las respuestas en el tiempo de cada uno de los filtros polifásicos

In [ ]:
#plt.figure()
for i in range(K):
    plt.subplots()
    #plt.stem(filts[i], linefmt='C'+str(i))
    plt.stem(Fp[i],use_line_collection=True)
    #plt.pause(1)
plt.show()

In [ ]:
for i in range(K):
    plt.subplots()
    plt.stem(Fp[i],use_line_collection=True)
    #plt.pause(1)
    plt.savefig('plots/f{}_impulse_resp_F.pdf'.format(i))
plt.show()

In [ ]:
#plt.figure()
for i in range(K):
    #plt.subplots()
    #mfreqz(K*Fp[i])
    h_freqz(K*Fp[i])
    #plt.savefig('plots/f{}.pdf'.format(i))
plt.show()

Primero el IFFT

In [ ]:
# IFFT (mixers) 
print('Run FFT....');
tx_fft_op_ideal = ifft(tx_data_in,axis=0)

In [ ]:
# FIR (filtros polifásicos)
tx_fir_op_ideal = np.zeros((K,data_len),dtype=complex)
print('Run FIRs....');
for n in range(K):
    tx_fir_op_ideal[n] = lfilter(Fp[n],1,tx_fft_op_ideal[n])

#tx_fir_op_ideal = np.flipud(tx_fir_op_ideal)

Guardamos los datos de salida del sintetizador.

In [ ]:
if write_golden_data:
    np.savetxt('../golden_data/tx_fir_op_ideal_real_{}.dat'.format(data_type),tx_fir_op_ideal.real, delimiter='\n', fmt='%.32e');
    np.savetxt('../golden_data/tx_fir_op_ideal_imag_{}.dat'.format(data_type),tx_fir_op_ideal.imag, delimiter='\n', fmt='%.32e');

Reshape en un solo stream de datos.

In [ ]:
print('Combine K....');
ant_ideal=np.reshape(tx_fir_op_ideal,(tx_fir_op_ideal.size),order='F')
print(ant_ideal.real.size)

Guardamos los datos de la antena.

In [ ]:
if write_golden_data:
    np.savetxt('../golden_data/ant_real_{}.dat'.format(data_type),ant_ideal.real, delimiter='\n', fmt='%.32e')
    np.savetxt('../golden_data/ant_imag_{}.dat'.format(data_type),ant_ideal.imag, delimiter='\n', fmt='%.32e')

Abro los archivos que tienen los datos de antena. c_frac_width es la parte fraccionaria de los datos.

In [ ]:
data_in=ant_ideal.real
#print(data_in[:10])
c_frac_width      = 15 # data fractional width

#ant_real = np.loadtxt('../golden_data/ant_real_cosine.dat',delimiter=',');
#ant_imag = np.loadtxt('../golden_data/ant_real_cosine.dat',delimiter=',');

f=open('../coefficients/coeff/ant_source_real_{}_{}_checks.coe'.format(data_type,K),'w')
#f.write('memory_initialization_radix=10;\n')
#f.write('memory_initialization_radix=16;\n')
#f.write('memory_initialization_vector=')
for i in range(ant_ideal.real.size):
    if (int(ant_ideal.real[i]*2**c_frac_width) < 0):
        f.write('{:x}'.format(int(round(ant_ideal.real[i]*2**c_frac_width+2**(c_frac_width+1)))))
    else:
        f.write('{:x}'.format(int(round(ant_ideal.real[i]*2**c_frac_width)+1)))
    if i == data_in.size-1:
        continue
        #f.write(';')
    else:
        f.write(',')
f.close()

f=open('../coefficients/coeff/ant_source_real_{}_{}.coe'.format(data_type,K),'w')
#f.write('memory_initialization_radix=10;\n')
f.write('memory_initialization_radix=16;\n')
f.write('memory_initialization_vector=')
for i in range(ant_ideal.real.size):
    if (int(ant_ideal.real[i]*2**c_frac_width) < 0):
        f.write('{:x}'.format(int(round(ant_ideal.real[i]*2**c_frac_width+2**(c_frac_width+1)))))
    else:
        f.write('{:x}'.format(int(round(ant_ideal.real[i]*2**c_frac_width)+1)))
    if i == data_in.size-1:
        f.write(';')
    else:
        f.write(',')
f.close()

data_in=ant_ideal.imag
#print(data_in[:10])
f=open('../coefficients/coeff/ant_source_imag_{}_{}_checks.coe'.format(data_type,K),'w')
#f.write('memory_initialization_radix=10;\n')
#f.write('memory_initialization_radix=16;\n')
#f.write('memory_initialization_vector=')
for i in range(ant_ideal.imag.size):
    if (int(ant_ideal.imag[i]*2**c_frac_width) < 0):
        f.write('{:x}'.format(int(round(ant_ideal.imag[i]*2**c_frac_width+2**(c_frac_width+1)))))
    else:
        f.write('{:x}'.format(int(round(ant_ideal.imag[i]*2**c_frac_width)+1)))
    if i == data_in.size-1:
        continue
        #f.write(';')
    else:
        f.write(',')
f.close()

f=open('../coefficients/coeff/ant_source_imag_{}_{}.coe'.format(data_type,K),'w')
#f.write('memory_initialization_radix=10;\n')
f.write('memory_initialization_radix=16;\n')
f.write('memory_initialization_vector=')
for i in range(ant_ideal.imag.size):
    if (int(ant_ideal.imag[i]*2**c_frac_width) < 0):
        f.write('{:x}'.format(int(round(ant_ideal.imag[i]*2**c_frac_width+2**(c_frac_width+1)))))
    else:
        f.write('{:x}'.format(int(round(ant_ideal.imag[i]*2**c_frac_width)+1)))
    if i == data_in.size-1:
        f.write(';')
    else:
        f.write(',')
f.close()

In [ ]:
fig,ax = plt.subplots(1,2,sharex=True, sharey=True, gridspec_kw={'wspace': 0.1})
fig.set_size_inches((10,8))
#fig.suptitle('Channelizer{}'.format(K),fontsize=18)

ax[0].plot(ant_ideal.real)
ax[0].set_ylabel(r'ch{} (V)'.format(n+1),fontsize=10)
ax[0].set_xlim(0,1024)
ax[0].grid()

ax[1].plot(ant_ideal.imag)
ax[1].set_xlim(0,1024)
ax[1].grid()
ax[0].set_title('Real',fontsize=12)
ax[1].set_title('Imaginario',fontsize=12)
#axs[0][1].set_title('Imaginary',fontsize=12)
ax[0].set_xlabel('n',fontsize=12)
ax[1].set_xlabel('n',fontsize=12)
#fig.align_xlabels()
plt.tight_layout()
#plt.savefig('plots/rx_fir_chann{}_out_1_{}.pdf'.format(K,int(K/2)))
#plt.show()

Plot output spectrum.

In [ ]:
print('Plot combined spectrum....');
fig,ax=plt.subplots(1,3,sharey=True,figsize=(10,5))
NFFT=K*data_len
f=fftfreq(NFFT,d=1/Fs)
#ww=kaiser(K*data_len,8);
ww = kaiser(NFFT, beta=8)
max_val   = max(abs(fft(ant_ideal*ww,NFFT)));
#max_val   = max(abs(fft(ant_ideal,NFFT)));
ax[0].plot(f/1e6,20*np.log10((abs(fft(ant_ideal*ww,NFFT)))/max_val),'b');
#ax.plot(f,20*np.log10((abs(fft(ant_ideal*ww,NFFT)))/max_val),'b');
#plt.plot(f,20*np.log10((abs(fft(ant_ideal,NFFT)))/max_val),'b');
for i in range(int(K/2)):
    ax[0].vlines((Fs/(1e6*K))*i,-120,0,linestyle='dotted',color='red')
for i in reversed(range(int(K/2))):
    ax[0].vlines(-(Fs/(1e6*K))*i,-120,0,linestyle='dotted',color='red')

#ax.vlines(-(Fs/K)*15+156.0e3,-150,0,color='green')
#ax.vlines(-(Fs/K)*15-180.0e3,-150,0,color='orange')
#plt.plot(f,20*np.log10(abs(fft(ant_ideal))/max_val),'b');
#ax.set_title('Espectro combinado del transmisor');#Transmitter combined spectrum');
#plt.legend('Ideal');
#axis tight
ax[0].axis([-Fs/(2*1e6), Fs/(2*1e6), -100, 5])
ax[0].set_ylabel('Amplitude (dB)',fontsize=12)
ax[0].set_xlabel('Frequency (MHz)',fontsize=12)
ax[0].legend(title='a)',title_fontsize='x-large', loc='upper right',frameon=True)#, fontsize='small'), ncol=2)

ax[1].plot(f/1e6,20*np.log10((abs(fft(ant_ideal*ww,NFFT)))/max_val),'b');
#ax.plot(f,20*np.log10((abs(fft(ant_ideal*ww,NFFT)))/max_val),'b');
#plt.plot(f,20*np.log10((abs(fft(ant_ideal,NFFT)))/max_val),'b');
for i in range(int(K/2)):
    ax[1].vlines((Fs/(K*1e6))*i,-120,0,linestyle='dotted',color='red')
for i in reversed(range(int(K/2))):
    ax[1].vlines(-(Fs/(K*1e6))*i,-120,0,linestyle='dotted',color='red')

channel=6
ax[1].axis([(channel-1)*(Fs/(K*1e6))-(Fs/(2*K*1e6)), (channel-1)*(Fs/(K*1e6))+(Fs/(2*K*1e6)), -100, 5])
#ax[1].set_ylabel('Amplitud (dB)',fontsize=12)
ax[1].set_xlabel('Frequency (MHz)',fontsize=12)
ax[1].legend(title='b)',title_fontsize='x-large', loc='upper right',frameon=True)

ax[2].plot(f/1e6,20*np.log10((abs(fft(ant_ideal*ww,NFFT)))/max_val),'b');
#ax.plot(f,20*np.log10((abs(fft(ant_ideal*ww,NFFT)))/max_val),'b');
#plt.plot(f,20*np.log10((abs(fft(ant_ideal,NFFT)))/max_val),'b');
for i in range(int(K/2)):
    ax[2].vlines((Fs/(K*1e6))*i,-120,0,linestyle='dotted',color='red')
for i in reversed(range(int(K/2))):
    ax[2].vlines(-(Fs/(K*1e6))*i,-120,0,linestyle='dotted',color='red')

ax[2].axis([(channel-1)*(Fs/(K*1e6))-(Fs/(32*K*1e6)), (channel-1)*(Fs/(K*1e6)+(Fs/(32*K*1e6))), -100, 5])
#ax[1].set_ylabel('Amplitud (dB)',fontsize=12)
ax[2].set_xlabel('Frequency (MHz)',fontsize=12)
ax[2].legend(title='c)',title_fontsize='x-large', loc='upper right',frameon=True)

plt.tight_layout()
plt.savefig('plots/zoom_tx_spectrum_en.pdf')

plt.show()

###########################################################################
# Receptor

A partir de acá implemento lo que sería el receptor
##########################################################################

Divido los datos en canales

El conmutador del receptor rota en sentido contrario al del transmisor. Por eso se usa la función "flipud".

Reshape de los coeficientes del filtro en filtros polifásicos es decir, dividir en las $K$ polifases. 

Acá es donde armo los filtros polifásicos: $\bar{p}_\rho(m) = h(mM-\rho),\quad \rho=0,1,...,M-1,\quad K=M$.
    
Todo esto, según pag. 361 de Crochiere y mis desarrollos en la pag. 186 de mi libro de notas.

In [ ]:
Ep=np.zeros((K,int(proto.size/K))) 

In [ ]:
for ro in range(K):
    for m in range(int(proto.size/K)):
        Ep[ro][m] = proto[int(M*m-ro)] 

In [ ]:
#for ro in range(1,K):
#    Ep[ro] = np.roll(Ep[ro],-1) 
for ro in range(1,K):
    Ep[ro][0] = 0 

Para graficar las respuestas en el tiempo de cada uno de los filtros polifásicos

In [ ]:
#plt.figure()
for i in range(K):
    #plt.subplots()
    #mfreqz(K*Ep[i])
    h_freqz(Ep[i])
    #plt.savefig('plots/f{}.pdf'.format(i))
    plt.tight_layout()

plt.show()

In [ ]:
#rx_data_in = np.flipud(np.reshape(ant_ideal,(K,data_len),order='F'));
rx_data_in = np.reshape(ant_ideal,(K,data_len),order='F');
#rx_data_in = np.reshape(ant_ideal,(K,data_len),order='F');

FIR (polyphase filters)

In [ ]:
print('Run FIRs....');
rx_fir_op_ideal = np.zeros((K,data_len),dtype=complex)
for n in range(K):
    rx_fir_op_ideal[n] = lfilter(Ep[n],1,rx_data_in[n])

In [ ]:
# Save FIR output
#if write_golden_data:
#    np.savetxt('../golden_data/rx_fir_op_real_{}.dat'.format(data_type),np.flipud(np.reshape(rx_fir_op_ideal.real,(K*data_len),order='F')),delimiter='\n',fmt='%.32e')
#    np.savetxt('../golden_data/rx_fir_op_imag_{}.dat'.format(data_type),np.flipud(np.reshape(rx_fir_op_ideal.imag,(K*data_len),order='F')),delimiter='\n',fmt='%.32e')
#    np.savetxt('../golden_data/ant_real_{}.dat'.format(data_type),ant_ideal.real, delimiter='\n', fmt='%.32e');
#    np.savetxt('../golden_data/ant_imag_{}.dat'.format(data_type),ant_ideal.imag, delimiter='\n', fmt='%.32e');

# Scale FIR output
# - The FIR output will be Fix19_19 but the IFFT input is quantized to Fix17_16 so
#   multiply FIR output by 2 to maximize precision to become Fix20_19
#rx_fir_op_core  = rx_fir_op_core * 2;
#rx_fir_op_ideal = rx_fir_op_ideal* 2;

Guardamos los datos de salida para comparar con tx_fir_op_ideal y lo que me dan las simulaciones con la FPGA.

In [ ]:
if write_golden_data:
    np.savetxt('../golden_data/rx_fir_op_ideal_real_{}.dat'.format(data_type),np.reshape(rx_fir_op_ideal.real,(K*data_len),order='F'), delimiter='\n', fmt='%.32e')
    np.savetxt('../golden_data/rx_fir_op_ideal_imag_{}.dat'.format(data_type),np.reshape(rx_fir_op_ideal.imag,(K*data_len),order='F'), delimiter='\n', fmt='%.32e')

IFFT (mixers). Es importante notar que antes filtré por K*Ep(i) y ahora le aplico la FFT. Esto en mi core no es así, ya que filtro por E(i)

In [ ]:
print('Run FFT....');
rx_fft_op_ideal = fft(rx_fir_op_ideal,axis=0)

#for n=1:data_len
#  [output, blkexp, overflow]=xfft_v8_0_bitacc_mex(rx_fft.generics, 3, quantize(rx_fft.q,rx_fir_op_core(:,n)), 0, 0); # Inverse FFT
#  rx_fft_op_core(:,n) = output.*rx_fft.output_scaling;
#end

# Save channel output data
#if write_golden_data
#  dlmwrite(['../golden_data/rx_chan_op_real_' data_type '.dat'],reshape(real(rx_fft_op_core),1,K*data_len),'delimiter', '\n','precision', '%.32e');
#  dlmwrite(['../golden_data/rx_chan_op_imag_' data_type '.dat'],reshape(imag(rx_fft_op_core),1,K*data_len),'delimiter', '\n','precision', '%.32e');
#end

Plot output spectrum

In [ ]:
fig,ax=plt.subplots(figsize=(8,6))
plot_type = 'spectrum';
#plot_type = 'time domain(abs)';
#plot_type = 'time domain(real)';

print('Plot output channel {}....'.format(plot_type));
# figure
ww=kaiser(data_len,8);
max_val = max(abs(fft(rx_fft_op_ideal[2]*ww)));#tener en cuenta que este canal debe estar presente
f=fftfreq(data_len,d=1/Fs)
for n in range(K):
    #chan_err = rx_fft_op_ideal(n,:) - rx_fft_op_core(n,:);
    if (plot_type =='spectrum'):
        ax.plot(fftshift(f),20*np.log10(abs(fftshift(fft(rx_fft_op_ideal[n]*ww)))/max_val),f'C{n}',label=f'ch{n}');
        ax.set_xlim(-Fs/2,Fs/2)
        ax.legend()
        #  plt.plot(20*np.log10(abs(fft(rx_fft_op_core[n]*ww))/max_val),'g');
        #  plt.plot(20*np.log10(abs(fft(chan_err*ww))/max_val),'r');
    elif (plot_type =='time domain(real)'):
        ax.plot(rx_fft_op_ideal[n][:1000].real,'C{}'.format(n));
        #  plt.plot(rx_fft_op_core[n].real,'g');
        #  plt.plot(abs(chan_err),'r');
    else:
        ax.plot(abs(rx_fft_op_ideal[n]),'C{}'.format(n));
        #  plt.plot(abs(rx_fft_op_core[n]),'g');
        #  plt.plot(abs(chan_err),'r');
        #ax.set_title('Receiver channel {:} {:}'.format(n,plot_type));
        #plt.legend('Ideal','Cores','Error');
#ax.set_xlim(0,300)
plt.tight_layout()
plt.show()

Ahora grafico lo que obtengo a la salida de los filtros polifásicos del Rx. Esto me va a servir para comparar con Vivado. Esto lo hago aparte en el script check_fir.py.

Grafico la salida de cada canal filtrado

In [ ]:
fig,axs = plt.subplots(int(K/2),2,sharex=True, sharey=True, gridspec_kw={'wspace': 0.1})
fig.set_size_inches((10,6))
#fig.suptitle('Channelizer{}'.format(K),fontsize=18)

for n in range(0,int(K/2)):
    ax=axs[n][0]
    ax.plot(rx_fir_op_ideal[n].real)
    #ax.plot(xc[n].real)
    #ax.plot(xc[n].real-64*tx_fir_op_ideal[n][:xc.size].real)
    #ax.plot(x_ideal[n::K].real,'r')
    #ax.legend(loc=0,fontsize=16)
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    #ax.set_ylabel(r'dB',fontsize=22)
    #ax.set_ylabel('ch{}[mV?]'.format(n+1),fontsize=14)
    ax.set_ylabel(r'ch{}'.format(n+1),fontsize=10)
    ax.set_xlim(0,1024)
    ax.grid()

    ax=axs[n][1]
    ax.plot(rx_fir_op_ideal[n].imag)
    #ax.plot(xc[n].imag)
    #ax.plot(xc[n].imag-64*tx_fir_op_ideal[n][:xc.size].imag)
    #ax.plot(x_ideal[n::K].imag,'r')
    #ax.legend(loc=0,fontsize=16)
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    #ax.set_ylabel(r'dB',fontsize=22)
    #ax.set_xlabel('ch{}'.format(n+1),fontsize=18)
    ax.set_xlim(0,1024)
    ax.grid()
    axs[0][0].set_title('Real',fontsize=12)
#axs[0][1].set_title('Imaginario',fontsize=18)
axs[0][1].set_title('Imaginary',fontsize=12)
axs[int(K/2)-1][0].set_xlabel('n',fontsize=12)
axs[int(K/2)-1][1].set_xlabel('n',fontsize=12)
#fig.align_xlabels()
plt.savefig(f'plots/rx_fir_chann{K}_out_1_{int(K/2)}.pdf')
#plt.show()

Los canales desde el 8 hasta el 15

In [ ]:
fig,axs = plt.subplots(int(K/2),2,sharex=True, sharey=True, gridspec_kw={'wspace': 0.1})
fig.set_size_inches((10,6))
#fig.suptitle('Channelizer{}'.format(K),fontsize=18)

for n in range(int(K/2),K):
    ax=axs[n-int(K/2)][0]
    ax.plot(rx_fir_op_ideal[n].real)
    #ax.plot(xc[n].real)
    #ax.plot(xc[n].real-64*tx_fir_op_ideal[n][:xc.size].real)
    #ax.plot(x_ideal[n::K].real)
    #ax.legend(loc=0,fontsize=16)
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    #ax.set_ylabel(r'dB',fontsize=22)
    #ax.set_ylabel('ch{}[mV?]'.format(n+1),fontsize=14)
    ax.set_ylabel('ch{}'.format(n+1),fontsize=10)
    ax.set_xlim(0,1024)
    ax.grid()

    ax=axs[n-int(K/2)][1]
    ax.plot(rx_fir_op_ideal[n].imag)
    #ax.plot(xc[n].imag)
    #ax.plot(xc[n].imag-64*tx_fir_op_ideal[n][:xc.size].imag)
    #ax.plot(x_ideal[n::K].imag)
    #ax.legend(loc=0,fontsize=16)
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    #ax.set_ylabel(r'dB',fontsize=22)
    #ax.set_xlabel('ch{}'.format(n+1),fontsize=18)
    ax.set_xlim(0,1024)
    ax.grid()

axs[0][0].set_title('Real',fontsize=12)
#axs[0][1].set_title('Imaginario',fontsize=18)
axs[0][1].set_title('Imaginary',fontsize=12)
axs[int(K/2)-1][0].set_xlabel('n',fontsize=12)
axs[int(K/2)-1][1].set_xlabel('n',fontsize=12)
#fig.align_xlabels()
plt.savefig(f'plots/rx_fir_chann{K}_out_{int(K/2+1)}_{K}.pdf')
plt.show()

Ahora grafico lo que obtengo a la salida del filtro polifásico del rx

Grafico la salida de cada canal filtrado

In [ ]:
fig,axs = plt.subplots(int(K/2),2,sharex=True, sharey=True, gridspec_kw={'wspace': 0.1})
fig.set_size_inches((10,8))
#fig.suptitle('Channelizer{}'.format(K),fontsize=18)

for n in range(0,int(K/2)):
    ax=axs[n][0]
    ax.plot(K**2*rx_fft_op_ideal[n].real)
    ax.set_ylabel(r'ch{}'.format(n+1),fontsize=10)
    ax.set_xlim(0,1024)
    ax.grid()

    ax=axs[n][1]
    ax.plot(K**2*rx_fft_op_ideal[n].imag)
    ax.set_xlim(0,1024)
    ax.grid()
    axs[0][0].set_title('Real (a.u.)',fontsize=12)
axs[0][1].set_title('Imaginario (a.u.)',fontsize=12)
#axs[0][1].set_title('Imaginary',fontsize=12)
axs[int(K/2)-1][0].set_xlabel('n',fontsize=12)
axs[int(K/2)-1][1].set_xlabel('n',fontsize=12)
#fig.align_xlabels()
plt.tight_layout()
plt.savefig('plots/dft_rxc{}_out_1_{}.pdf'.format(K,int(K/2)))
#plt.show()

In [ ]:
#Grafico la salida de cada canal
fig,axs = plt.subplots(int(K/2),2,sharex=True, sharey=True, gridspec_kw={'wspace': 0.1})
fig.set_size_inches((10,8))
#fig.suptitle('Channelizer{}'.format(K),fontsize=18)

for n in range(int(K/2),K):
    ax=axs[n-int(K/2)][0]
    ax.plot(K**2*rx_fft_op_ideal[n].real)
    ax.set_ylabel('ch{}'.format(n+1),fontsize=10)
    ax.set_xlim(0,1024)
    ax.grid()

    ax=axs[n-int(K/2)][1]
    ax.plot(K**2*rx_fft_op_ideal[n].imag)
    ax.set_xlim(0,1024)
    ax.grid()

axs[0][0].set_title('Real (a.u.)',fontsize=12)
axs[0][1].set_title('Imaginario (a.u.)',fontsize=12)
#axs[0][1].set_title('Imaginary',fontsize=12)
axs[int(K/2)-1][0].set_xlabel('n',fontsize=12)
axs[int(K/2)-1][1].set_xlabel('n',fontsize=12)
#fig.align_xlabels()
plt.tight_layout()
plt.savefig('plots/dft_rxc{}_out_{}_{}.pdf'.format(K,int(K/2+1),K))
plt.show()

Esta ultima parte es para comparar los datos obtenidos de la Redpitaya con las simulaciones. Lo que sigue lo tomo del script in_gen.ipynb

In [ ]:
import os
fname='test02.dat'
#proc_dir ='/home/arnaldi/seagate2tb/data/mkid/rp/dft_rxc16/'
proc_dir ='/home/arnaldi/work//data/dft_rxc16/'

In [ ]:
c1,c2 = np.loadtxt(os.path.join(proc_dir, fname), unpack=1, skiprows=19)#, usecols=(0,1,3))
#c11,c22 = h_loadtxt(os.path.join(proc_dir, fname), ncols=2)#, usecols=(0,1,3))
c1=np.resize(c1,c1.size - len(c1)%K)
c2=np.resize(c2,c2.size - len(c2)%K)
data_len=c1.size

In [ ]:
idata=np.reshape(c1,(K,int(c1.size/K)),order='F')
qdata=np.reshape(c2,(K,int(c2.size/K)),order='F')

data=np.zeros((K,idata.shape[1]),dtype=complex)                                                
print(idata.shape)


#armo los canales
for i in range(K): 
    for j in range(idata.shape[1]): 
        data[i][j]=idata[i][j]+1j*qdata[i][j] 

#data=32*data

data_len=data.shape[1]

In [ ]:
fig,axs = plt.subplots(int(K/2),2,sharex=True, sharey=True, gridspec_kw={'wspace': 0.1})
fig.set_size_inches((10,8))
#fig.suptitle('Channelizer{}'.format(K),fontsize=18)

for n in range(0,int(K/2)):
    ax=axs[n][0]
    ax.plot((4*rx_fft_op_ideal[n].real)*1e3,'--')
    ax.plot((data[n].real/2**13)*1e3)
    ax.set_ylabel(r'ch{}'.format(n+1),fontsize=10)
    ax.set_xlim(0,2**10)
    ax.grid()
    #ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

    ax=axs[n][1]
    ax.plot((4*rx_fft_op_ideal[n].imag)*1e3,'--')
    ax.plot((data[n].imag/2**13)*1e3)
    ax.set_xlim(0,2**10)
    ax.grid()
    #ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    
axs[0][0].set_title('Real (mV)',fontsize=12)
axs[0][1].set_title('Imaginario (mV)',fontsize=12)
#axs[0][1].set_title('Imaginary',fontsize=12)
axs[int(K/2)-1][0].set_xlabel('n',fontsize=12)
axs[int(K/2)-1][1].set_xlabel('n',fontsize=12)
#fig.align_xlabels()
#plt.tight_layout()
plt.savefig('plots/dft_rxc{}_out_1_{}_compa.pdf'.format(K,int(K/2)))
#plt.show()

In [ ]:
#Grafico la salida de cada canal
fig,axs = plt.subplots(int(K/2),2,sharex=True, sharey=True, gridspec_kw={'wspace': 0.1})
fig.set_size_inches((10,8))
#fig.suptitle('Channelizer{}'.format(K),fontsize=18)

for n in range(int(K/2),K):
    ax=axs[n-int(K/2)][0]
    ax.plot((4*rx_fft_op_ideal[n].real)*1e3,'--')
    ax.plot((data[n].real/2**13)*1e3)
    ax.set_ylabel('ch{}'.format(n+1),fontsize=10)
    ax.set_xlim(0,2**10)
    ax.grid()
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

    ax=axs[n-int(K/2)][1]
    ax.plot((4*rx_fft_op_ideal[n].imag)*1e3,'--')
    ax.plot((data[n].imag/2**13)*1e3)
    ax.set_xlim(0,2**10)
    ax.grid()
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

axs[0][0].set_title('Real (mV)',fontsize=12)
axs[0][1].set_title('Imaginario (mV)',fontsize=12)
#axs[0][1].set_title('Imaginary',fontsize=12)
axs[int(K/2)-1][0].set_xlabel('n',fontsize=12)
axs[int(K/2)-1][1].set_xlabel('n',fontsize=12)
#fig.align_xlabels()
#plt.tight_layout()
plt.savefig('plots/dft_rxc{}_out_{}_{}_compa.pdf'.format(K,int(K/2+1),K))
plt.show()
print(data_len)

In [ ]:
fig,ax=plt.subplots(2,1,sharex=True, figsize=(8,5))
plot_type = 'spectrum';
#plot_type = 'time domain(abs)';
#plot_type = 'time domain(real)';
NFFT=2**16
print(rx_fft_op_ideal.shape[1])
print('Plot output channel {}....'.format(plot_type));
#simulacion
wws=kaiser(rx_fft_op_ideal.shape[1],8);
max_vals = max(abs(fft(rx_fft_op_ideal[2]*wws,NFFT)));#tener en cuenta que este canal debe estar presente
fss=fftfreq(NFFT,d=1/(Fs))
# figure
ww=kaiser(data.shape[1],8);
max_val = max(abs(fft(data[2]*ww,NFFT)));#tener en cuenta que este canal debe estar presente
f=fftfreq(NFFT,d=1/Fs)
for n in range(K):
    #chan_err = rx_fft_op_ideal(n,:) - rx_fft_op_core(n,:);
    if (plot_type =='spectrum'):
        ax[0].plot(fftshift(fss),20*np.log10(abs(fftshift(fft(rx_fft_op_ideal[n]*wws,NFFT)))/max_vals),f'C{n}',label=f'ch{n}');
        ax[1].plot(fftshift(f),20*np.log10(abs(fftshift(fft(data[n]*ww,NFFT)))/max_val),f'C{n}',label=f'ch{n}');
        #ax.set_xlim(-Fs/(2*K),Fs/(2*K))
        ax[0].set_xlim(0,Fs/2)
        ax[1].set_xlim(0,Fs/2)
        ax[0].set_ylim(-95,5)
        ax[1].set_ylim(-95,5)
        #ax.legend(loc='lower center',ncol=16)
        ax[0].legend(loc='upper right', fontsize='small', ncol=4)
        ax[1].legend(loc='upper right', fontsize='small', ncol=4)
        ax[0].set_ylabel('Amplitud (dB)',fontsize=12)
        ax[1].set_ylabel('Amplitud (dB)',fontsize=12)
        #ax[0].set_ylabel(r'Amplitude (dB)',fontsize=12)
        #ax[0].set_xlabel('Frequency (Hz)',fontsize=12)
        #ax[1].set_ylabel(r'Amplitude (dB)',fontsize=12)
        #ax[1].set_xlabel('Frequency (Hz)',fontsize=12)
        ax[1].set_xlabel('Frecuencia (Hz)',fontsize=12)
        ax[0].grid()
        ax[1].grid()
        #  plt.plot(20*np.log10(abs(fft(rx_fft_op_core[n]*ww))/max_val),'g');
        #  plt.plot(20*np.log10(abs(fft(chan_err*ww))/max_val),'r');
    elif (plot_type =='time domain(real)'):
        ax.plot(data[n][:1000].real,'C{}'.format(n));
        #  plt.plot(rx_fft_op_core[n].real,'g');
        #  plt.plot(abs(chan_err),'r');
    else:
        ax.plot(abs(data[n]),'C{}'.format(n));
        #  plt.plot(abs(rx_fft_op_core[n]),'g');
        #  plt.plot(abs(chan_err),'r');
        #ax.set_title('Receiver channel {:} {:}'.format(n,plot_type));
        #plt.legend('Ideal','Cores','Error');
#ax.set_xlim(0,300)
plt.tight_layout()
plt.savefig('plots/dft_rxc16_compa_sim_data.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(2,1,sharex=True,figsize=(8,5))
plot_type='spectrum'
NFFT=2**16 #data.shape[1]
#simulacion
wws=kaiser(rx_fft_op_ideal.shape[1],8);
max_vals = max(abs(fft(rx_fft_op_ideal[2]*wws,NFFT)));#tener en cuenta que este canal debe estar presente
fss=fftfreq(NFFT,d=1/Fs)
# figure
NFFTi=2**16 #data.shape[1]
ww=kaiser(data.shape[1],8);
max_val = max(abs(fft(data[2]*ww,NFFTi)));#tener en cuenta que este canal debe estar presente
f=fftfreq(NFFTi,d=1/Fs)
for n in range(K):
    #chan_err = rx_fft_op_ideal(n,:) - rx_fft_op_core(n,:);
    if (plot_type =='spectrum'):
        ax[0].plot(fftshift(fss),20*np.log10(abs(fftshift(fft(rx_fft_op_ideal[n]*wws,NFFT)))/max_vals),f'C{n}',label=f'ch{n}');
        ax[1].plot(fftshift(f),20*np.log10(abs(fftshift(fft(data[n]*ww,NFFTi)))/max_val),f'C{n}',label=f'ch{n}');
        #ax.set_xlim(-Fs/(2*K),Fs/(2*K))
        ax[0].grid()
        ax[0].set_xlim(0,Fs/2)
        ax[1].set_xlim(0,Fs/2)
        ax[0].set_ylim(-95,5)
        ax[1].set_ylim(-95,5)
        #ax[0].set_ylabel('Amplitud (dB)')
        #ax[1].set_ylabel('Amplitud (dB)')
        #ax.legend(loc='lower center',ncol=16)
        ax[0].legend(loc='upper right', fontsize='small', ncol=4)
        ax[1].legend(loc='upper right', fontsize='small', ncol=4)
        #ax[0].set_ylabel('Amplitud (dB)')
        #ax[1].set_ylabel('Amplitud (dB)')
        ax[0].set_ylabel(r'Amplitude (dB)',fontsize=12)
        #ax[0].set_xlabel('Frequency (Hz)',fontsize=12)
        ax[1].set_ylabel(r'Amplitude (dB)',fontsize=12)
        ax[1].set_xlabel('Frequency (Hz)',fontsize=12)
        #  plt.plot(20*np.log10(abs(fft(rx_fft_op_core[n]*ww))/max_val),'g');
        #  plt.plot(20*np.log10(abs(fft(chan_err*ww))/max_val),'r');
        
    elif (plot_type =='time domain(real)'):
        ax.plot(data[n][:1000].real,'C{}'.format(n));
        #  plt.plot(rx_fft_op_core[n].real,'g');
        #  plt.plot(abs(chan_err),'r');
    else:
        ax.plot(abs(data[n]),'C{}'.format(n));
        #  plt.plot(abs(rx_fft_op_core[n]),'g');
        #  plt.plot(abs(chan_err),'r');
        #ax.set_title('Receiver channel {:} {:}'.format(n,plot_type));
        #plt.legend('Ideal','Cores','Error');
#ax.set_xlim(0,300)
ax[0].grid()
ax[0].set_xlim(0,Fs/2)
ax[1].set_xlim(0,Fs/2)
ax[0].set_ylim(-95,5)
ax[1].set_ylim(-95,5)
#ax[0].set_ylabel('Amplitud (dB)')
#ax[1].set_ylabel('Amplitud (dB)')
#ax.legend(loc='lower center',ncol=16)
ax[0].legend(loc='upper right', fontsize='small', ncol=2)
ax[1].legend(loc='upper right', fontsize='small', ncol=2)
#ax[0].set_ylabel('Amplitud (dB)')
#ax[1].set_ylabel('Amplitud (dB)')
ax[0].set_ylabel(r'Amplitude (dB)',fontsize=12)
#ax[0].set_xlabel('Frequency (Hz)',fontsize=12)
ax[1].set_ylabel(r'Amplitude (dB)',fontsize=12)
ax[1].set_xlabel('Frequency (Hz)',fontsize=12)
#  plt.plot(20*np.log10(abs(fft(rx_fft_op_core[n]*ww))/max_val),'g');
#  plt.plot(20*np.log10(abs(fft(chan_err*ww))/max_val),'r');
plt.grid()

# this is an inset axes over the main axes
a = plt.axes([.4, .4, .3, .3], alpha=0.02)
for i in range(K):
    a.plot(fftshift(fss),20*np.log10(abs(fftshift(fft(rx_fft_op_ideal[i]*wws,NFFT)))/max_vals),f'C{i}',label=f'ch{i}');
    #a.plot(f[:int(N/2)],X[i])
    #a.plot(fftshift(f),20*np.log10(fftshift(X[i])),label='CH{}'.format(i+1),lw=1)
#for i in range(0,len(PX)):
#    a.plot(f[:int(N/2)],PX[i])
a.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
#a.set_xlabel('Charge [ADC.bin]',fontsize=14, fontname='monospace')
#a.set_ylabel('Counts',fontsize=14, fontname='monospace')
a.set_xlim(0,Fs/K)
a.set_ylim(-75,2)
#a.autoscale()
a.grid()

plt.tight_layout()
plt.savefig('plots/dft_rxc16_compa_sim_data.pdf')
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

fig,ax=plt.subplots(2,1,sharex=True,figsize=(10,8))
NFFT=2**16 #data.shape[1]
#simulacion
wws=kaiser(rx_fft_op_ideal.shape[1],8);
max_vals = max(abs(fft(rx_fft_op_ideal[2]*wws,NFFT)));#tener en cuenta que este canal debe estar presente
fss=fftfreq(NFFT,d=1/Fs)
# figure
NFFTi=2**16 #data.shape[1]
ww=kaiser(data.shape[1],8);
max_val = max(abs(fft(data[2]*ww,NFFTi)));#tener en cuenta que este canal debe estar presente
f=fftfreq(NFFTi,d=1/Fs)
for n in range(K):
    ax[0].plot(fftshift(fss)/1e6,20*np.log10(abs(fftshift(fft(rx_fft_op_ideal[n]*wws,NFFT)))/max_vals),f'C{n}',label=f'ch{n}');    
    ax[1].plot(fftshift(f)/1e6,20*np.log10(abs(fftshift(fft(data[n]*ww,NFFTi)))/max_val),f'C{n}',label=f'ch{n}');
    
#ax.set_xlim(-Fs/(2*K),Fs/(2*K))   
ax[1].set_xlim(0,Fs/(2*1e6))
ax[1].set_ylim(-95,5)
#ax[1].set_ylabel('Amplitud (dB)')
#ax.legend(loc='lower center',ncol=16)    
ax[1].legend(title='b)',title_fontsize='x-large', loc='upper right', fontsize='small', ncol=2)    
#ax[1].set_ylabel('Amplitud (dB)')    
ax[1].set_ylabel(r'Amplitude (dB)',fontsize=12)
ax[1].set_xlabel('Frequency (MHz)',fontsize=12)
#  plt.plot(20*np.log10(abs(fft(rx_fft_op_core[n]*ww))/max_val),'g');
#  plt.plot(20*np.log10(abs(fft(chan_err*ww))/max_val),'r');
    
ax[0].legend(title='a)',title_fontsize='x-large', loc='upper right', fontsize='small', ncol=2)
#ax[0].set_ylabel('Amplitud (dB)')
ax[0].set_ylabel(r'Amplitude (dB)',fontsize=12)
#ax[0].set_xlabel('Frequency (Hz)',fontsize=12)
    
#ax.set_xlim(0,300)
ax[0].grid()
ax[0].set_xlim(0,Fs/(2*1e6))
#ax[1].set_xlim(0,Fs/2)
ax[0].set_ylim(-95,5)
#ax[1].set_ylim(-95,5)
#ax[0].set_ylabel('Amplitud (dB)')
#ax[1].set_ylabel('Amplitud (dB)')
#ax.legend(loc='lower center',ncol=16)
#ax[0].legend(loc='upper right', fontsize='small', ncol=2)
#ax[1].legend(loc='upper right', fontsize='small', ncol=2)
#ax[0].set_ylabel('Amplitud (dB)')
#ax[1].set_ylabel('Amplitud (dB)')
ax[0].set_ylabel(r'Amplitude (dB)',fontsize=12)
#ax[0].set_xlabel('Frequency (Hz)',fontsize=12)
ax[1].set_ylabel(r'Amplitude (dB)',fontsize=12)
ax[1].set_xlabel('Frequency (MHz)',fontsize=12)
#  plt.plot(20*np.log10(abs(fft(rx_fft_op_core[n]*ww))/max_val),'g');
#  plt.plot(20*np.log10(abs(fft(chan_err*ww))/max_val),'r');
plt.grid()


# Create inset of width 1.3 inches and height 0.9 inches
# at the default upper right location
#axins = inset_axes(ax[0], width=1.3, height=0.9)

# Create inset of width 30% and height 40% of the parent axes' bounding box
# at the lower left corner (loc=3)
#axins2 = inset_axes(ax[0], width="30%", height="40%", loc=3)

# Create inset of mixed specifications in the second subplot;
# width is 30% of parent axes' bounding box and
# height is 1 inch at the upper left corner (loc=2)
a = inset_axes(ax[0], 
               width="50%", 
               height="100%", 
               bbox_to_anchor=(.3, .43, .8, .55),
               bbox_transform=ax[0].transAxes,
               loc=3)
for i in range(K):
    a.plot(fftshift(fss)/1e6,20*np.log10(abs(fftshift(fft(rx_fft_op_ideal[i]*wws,NFFT)))/max_vals),f'C{i}',label=f'ch{i}');
    #a.plot(f[:int(N/2)],X[i])
#a.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
a.tick_params(labelsize=10)
a.set_xlim(0,5)
a.set_ylim(-95,5)
#a.autoscale()
a.grid()

b = inset_axes(ax[1], 
               width="50%", 
               height="100%", 
               bbox_to_anchor=(.3, .43, .8, .55),
               bbox_transform=ax[1].transAxes,
               loc=3)
for i in range(K):
    b.plot(fftshift(f)/1e6,20*np.log10(abs(fftshift(fft(data[i]*ww,NFFTi)))/max_val),f'C{i}',label=f'ch{i}');    #a.plot(f[:int(N/2)],X[i])
#b.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
b.tick_params(labelsize=10)
b.set_xlim(0,5)
b.set_ylim(-95,5)
#a.autoscale()
b.grid()
# Create an inset in the lower right corner (loc=4) with borderpad=1, i.e.
# 10 points padding (as 10pt is the default fontsize) to the parent axes
#axins4 = inset_axes(ax[1], width="20%", height="20%", loc=4, borderpad=1)

# Turn ticklabels of insets off
#for axi in [axins, axins2, axins3, axins4]:
#    axi.tick_params(labelleft=False, labelbottom=False)


# this is an inset axes over the main axes
#a = plt.axes(ax[0], alpha=0.02)
#for i in range(K):
#    a.plot(fftshift(fss),20*np.log10(abs(fftshift(fft(rx_fft_op_ideal[i]*wws,NFFT)))/max_vals),f'C{i}',label=f'ch{i}');
    #a.plot(f[:int(N/2)],X[i])
    #a.plot(fftshift(f),20*np.log10(fftshift(X[i])),label='CH{}'.format(i+1),lw=1)
#for i in range(0,len(PX)):
#    a.plot(f[:int(N/2)],PX[i])
#a.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
#a.set_xlabel('Charge [ADC.bin]',fontsize=14, fontname='monospace')
#a.set_ylabel('Counts',fontsize=14, fontname='monospace')
#a.set_xlim(0,Fs/K)
#a.set_ylim(-75,2)
#a.autoscale()
#a.grid()

plt.tight_layout()
plt.savefig('plots/dft_rxc16_compa_sim_data_inset.pdf')
plt.show()